In [103]:
import pathlib
import re
import time 
import itertools

from selenium import webdriver

import numpy as np
import pandas as pd

In [104]:
CURRENT_PATH = pathlib.Path().absolute()
PATH = str(CURRENT_PATH) + '\\chromedriver.exe'
driver = webdriver.Chrome(PATH)

In [105]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

def check_exists_by_class(class_name):
    try:
        driver.find_element_by_class_name(class_name)
    except NoSuchElementException:
        return False
    return True

## Loop proces

In [106]:
# Hard pause between pages - due to github number of request per minute limitation
WAIT_TIME_STEP = 0.5
data = []

# Wait for loading element
driver.implicitly_wait(10)

# ----------------------------------------
## Login
driver.get('https://github.com/login?return_to=%2Fcollections')
time.sleep(WAIT_TIME_STEP)
login_input = driver.find_element_by_id("login_field")
# Fill Login Name
login_input.send_keys('')

# Password
password_input = driver.find_element_by_id("password")
# Fill Pawwrod
password_input.send_keys('')

# Submit
submit_btn = driver.find_element_by_name("commit")
submit_btn.click()
time.sleep(WAIT_TIME_STEP)
# ----------------------------------------


driver.get('https://github.com/collections')
if(check_exists_by_xpath("//button[contains(@data-disable-with,'Loading more')]")):
    driver.find_element_by_xpath("//button[contains(@data-disable-with,'Loading more')]").click()
    time.sleep(WAIT_TIME_STEP)
collections = driver.find_elements_by_tag_name('article')
print("Total number of Collections", len(collections))

for collection_index in range(36,len(collections),1):
    # Collection List
    # Relaunch catalog of collections and pick another collection to gather
    
    driver.get('https://github.com/collections')
    if(check_exists_by_xpath("//button[contains(@data-disable-with,'Loading more')]")):
                driver.find_element_by_xpath("//button[contains(@data-disable-with,'Loading more')]").click()
    time.sleep(WAIT_TIME_STEP*2)
    collection = driver.find_elements_by_tag_name('article')[collection_index]
    
    # ----------------------------------------
    # SCRAPING - List of Collections
    collection_link = collection.find_element_by_tag_name('a')
    collection_href = collection_link.get_attribute('href')
    collection_title = collection_link.text
    print('Collection:', collection_title)
    # ----------------------------------------
    
    # Go inside a collection
    driver.execute_script("arguments[0].click();", collection_link)
    time.sleep(WAIT_TIME_STEP)
    
    # Collection
    if(check_exists_by_xpath("//button[contains(@data-disable-with,'Loading more')]")):
        driver.find_element_by_xpath("//button[contains(@data-disable-with,'Loading more')]").click()
        time.sleep(WAIT_TIME_STEP*2)
    
    repo_list = driver.find_elements_by_tag_name('article')
    print('Number of repos in the collection: ', len(repo_list))
    
    for repo_list_index in range(len(repo_list)):
        # Starting
        if(repo_list_index != 0):
            driver.get(collection_href)
            if(check_exists_by_xpath("//button[contains(@data-disable-with,'Loading more')]")):
                driver.find_element_by_xpath("//button[contains(@data-disable-with,'Loading more')]").click()
                time.sleep(WAIT_TIME_STEP*2)
            

        repo_list_item = driver.find_elements_by_tag_name('article')[repo_list_index]
        # filter out video items
        if('video' not in repo_list_item.find_element_by_tag_name('div').text.lower()):
            
            num_stars = None
            num_forks = None
            
            # ----------------------------------------
            # SCRAPING - List of repositories
            for repo_a in repo_list_item.find_elements_by_tag_name('a'):
                if('stargazers' in repo_a.get_attribute('href').lower()):
                    num_stars=repo_a.text
                    
                if('members' in repo_a.get_attribute('href').lower()):
                    num_forks=repo_a.text
             # ----------------------------------------
            
            repo_list_item_h1 = repo_list_item.find_element_by_tag_name('h1')
            repo_list_link = repo_list_item_h1.find_element_by_tag_name('a')
            repo_list_title = repo_list_link.text
            print('Repo:', repo_list_title)
           
            

            # Click to Repo detail page
            driver.execute_script("arguments[0].click();", repo_list_link)
            #repo_list_link.click()
            #wait.until(ec.visibility_of_element_located((By.XPATH, "//a[@data-tracking-id='men']")))
            time.sleep(WAIT_TIME_STEP * 2)
            
            # ----------------------------------------
            # SCRAPING - Repository data
            
            repo_name = driver.find_element_by_xpath("//strong[contains(@itemprop,'name')]").text
            repo_author = driver.find_element_by_xpath("//span[contains(@itemprop,'author')]").text
            
            # Top right bar
            page_head = driver.find_element_by_class_name('pagehead-actions')
            # Watchers
            watch = page_head.find_elements_by_xpath("//a[contains(@href,'watchers')]")[0]
            num_watchers_dirty = watch.text
            if('k' in num_watchers_dirty):
                num_watchers_dirty = num_watchers_dirty.replace('.','').replace('k','00')
            num_watchers = num_watchers_dirty
            # Dependents
            num_dependents = None
            if(check_exists_by_xpath("//a[contains(@href,'/dependents')]")):
                dependents = page_head.find_element_by_xpath("//a[contains(@href,'/dependents')]")
                num_dependents_dirty = dependents.text
                if('k' in num_dependents_dirty):
                    num_dependents_dirty = num_dependents_dirty.replace('.','').replace('k','00')
                num_dependents = num_dependents_dirty
            
            # Summary middle bar
            summary_bar = driver.find_element_by_class_name('numbers-summary')
            commmit = summary_bar.find_elements_by_tag_name('li')[0]
            branches = summary_bar.find_elements_by_tag_name('li')[1]
            packages = summary_bar.find_elements_by_tag_name('li')[2]
            releases = summary_bar.find_elements_by_tag_name('li')[3]
            contributors = summary_bar.find_elements_by_tag_name('li')[4]

            # Commits
            num_commits = commmit.find_element_by_class_name('num').text.replace(',','')
            # Branches
            num_branches = branches.find_element_by_class_name('num').text.replace(',','')
            # Packages
            num_packages = packages.find_element_by_class_name('num').text.replace(',','')
            # Releases
            num_releases = releases.find_element_by_class_name('num').text.replace(',','')
            # Contributors
            num_contributors = contributors.find_element_by_class_name('num').text.replace(',','')
            
            # Language Usage
            lang_usage_dict = {}
            if(check_exists_by_class('repository-lang-stats-graph')):
                lang_bar = driver.find_element_by_class_name('repository-lang-stats-graph')
                lang_stats = lang_bar.find_elements_by_tag_name('span')

                for lang_item in lang_stats:
                    lang_info = lang_item.get_attribute('aria-label')
                    lang_name = ["".join(x) for _, x in itertools.groupby(lang_info, key=str.isdigit)][0].strip()
                    lang_usage = ["".join(x) for _, x in itertools.groupby(lang_info, key=str.isdigit)][1].replace('%','')
                    lang_usage = float(lang_usage)
                    lang_usage_dict[lang_name] = lang_usage
                

            issues_open_num = None
            issues_closed_num = None
            # GO TO issues
            if(check_exists_by_xpath("//a[contains(@data-selected-links,'repo_issues')]")):
                issues_link = driver.find_element_by_xpath("//a[contains(@data-selected-links,'repo_issues')]")
                driver.execute_script("arguments[0].click();", issues_link)
                #issues_link.click()
                time.sleep(WAIT_TIME_STEP)

                ## Collect Issues
                issues = driver.find_elements_by_css_selector(".table-list-header-toggle")[0]
                issues_open = issues.find_elements_by_tag_name('a')[0].get_attribute('textContent')
                issues_open_num = ''.join(re.findall(r'\d+', issues_open))

                # Issues Closed
                issues_closed = issues.find_elements_by_tag_name('a')[1].get_attribute('textContent')
                issues_closed_num = ''.join(re.findall(r'\d+', issues_closed))
            
            # GO TO Pull requests
            pull_request_open_num = None
            pull_request_closed_num = None
            if(check_exists_by_xpath("//a[contains(@data-selected-links,'repo_pulls')]")):
                pull_request_link = driver.find_element_by_xpath("//a[contains(@data-selected-links,'repo_pulls')]")
                pull_request_link.click()
                time.sleep(WAIT_TIME_STEP)

                # Pull requests
                pull_request = driver.find_elements_by_css_selector(".table-list-header-toggle")[0]
                pull_request_open = pull_request.find_elements_by_tag_name('a')[0].get_attribute('textContent')
                pull_request_open_num = ''.join(re.findall(r'\d+', pull_request_open))

                # Pull requests Closed
                pull_request_closed = pull_request.find_elements_by_tag_name('a')[1].get_attribute('textContent')
                pull_request_closed_num = ''.join(re.findall(r'\d+', pull_request_closed))
            # ----------------------------------------
            
            collected_repo_data = [collection_title, repo_name, repo_author, num_stars, num_forks, 
                                   num_watchers,num_dependents, num_commits, num_branches,num_packages, num_releases, 
                                   num_contributors, issues_open_num, issues_closed_num, pull_request_open_num,
                                   pull_request_closed_num, lang_usage_dict]
            data.append(collected_repo_data)
   
    

Total number of Collections 40
Collection: Code Quality Checker Tools For PHP Projects
Number of repos in the collection:  10
Repo: squizlabs / PHP_CodeSniffer
Repo: FriendsOfPHP / PHP-CS-Fixer
Repo: psecio / parse
Repo: phan / phan
Repo: sebastianbergmann / phpcpd
Repo: sebastianbergmann / phploc
Repo: povils / phpmnd
Repo: phpmd / phpmd
Repo: vimeo / psalm
Repo: infection / infection
Collection: Game Engines
Number of repos in the collection:  20
Repo: godotengine / godot
Repo: turbulenz / turbulenz_engine
Repo: GarageGames / Torque3D
Repo: GarageGames / Torque2D
Repo: spring / spring
Repo: cocos2d / cocos2d-x
Repo: Gamua / Starling-Framework
Repo: gameplay3d / gameplay
Repo: jMonkeyEngine / jmonkeyengine
Repo: SFTtech / openage
Repo: MonoGame / MonoGame
Repo: libgdx / libgdx
Repo: superpowers / superpowers-core
Repo: AtomicGameEngine / AtomicGameEngine
Repo: 4ian / GDevelop
Repo: CRYTEK / CRYENGINE
Repo: urho3d / Urho3D
Repo: benoit-dumas / OpenRTS
Repo: photonstorm / phaser
Repo: m

In [107]:
data

[['Code Quality Checker Tools For PHP Projects',
  'PHP_CodeSniffer',
  'squizlabs',
  '7691',
  '1240',
  '204',
  '67200',
  '6369',
  '6',
  '0',
  '79',
  '179',
  '193',
  '1633',
  '55',
  '1098',
  {'PHP': 97.0, 'JavaScript': 1.0, 'Other': 0.0}],
 ['Code Quality Checker Tools For PHP Projects',
  'PHP-CS-Fixer',
  'FriendsOfPHP',
  '9047',
  '1136',
  '221',
  '22800',
  '6369',
  '4',
  '0',
  '156',
  '229',
  '255',
  '1579',
  '79',
  '3078',
  {'PHP': 99.0, 'Shell': 0.0}],
 ['Code Quality Checker Tools For PHP Projects',
  'parse',
  'psecio',
  '289',
  '39',
  '21',
  '',
  '359',
  '2',
  '0',
  '8',
  '6',
  '15',
  '30',
  '3',
  '50',
  {'PHP': 100.0}],
 ['Code Quality Checker Tools For PHP Projects',
  'phan',
  'phan',
  '4827',
  '320',
  '115',
  '786',
  '6072',
  '9',
  '0',
  '129',
  '88',
  '590',
  '1518',
  '3',
  '1822',
  {'PHP': 99.0, 'Other': 0.0}],
 ['Code Quality Checker Tools For PHP Projects',
  'phpcpd',
  'sebastianbergmann',
  '1950',
  '158',
  

## Transformace a export dat

In [108]:
labels= ['collection', 'repository','author','stars','forks','watchers', 'dependents', 'commits', 'branches'
 , 'packages', 'releases', 'contributors', 'issues_open', 'issues_closed', 'pull_requests_open', 'pull_requests_closed'
 ,'lang_used']
df = pd.DataFrame(data, columns=labels)
df.tail()

,collection,author,repository,stars,forks,watchers,dependents,commits,branches,packages,releases,contributors,issues_open,issues_closed,pull_requests_open,pull_requests_closed,lang_used
52,Static Site Generators,react-static,react-static,8715,734,129,2200,1541,7,0,348,209,69,759,3,601,"{'JavaScript': 92.0, 'TypeScript': 4.0, 'CSS':..."
53,Static Site Generators,docsify,docsifyjs,13807,3233,227,1100,1286,13,0,185,141,106,693,20,372,"{'JavaScript': 82.0, 'CSS': 13.0, 'HTML': 3.0,..."
54,Static Site Generators,mkdocs,mkdocs,10156,1489,227,6600,1457,6,0,44,147,94,1234,16,773,"{'Python': 71.0, 'JavaScript': 21.0, 'HTML': 5..."
55,Static Site Generators,harp,sintaxi,4868,377,115,,638,83,0,91,28,21,518,19,90,"{'JavaScript': 84.0, 'HTML': 12.0, 'CSS': 2.0,..."
56,Static Site Generators,netlify-cms,netlify,11841,1774,270,41200,2853,34,0,1219,320,465,1367,7,1997,"{'JavaScript': 68.0, 'TypeScript': 24.0, 'HTML..."


In [109]:
df.dtypes

collection              object
author                  object
repository              object
stars                   object
forks                   object
watchers                object
dependents              object
commits                 object
branches                object
packages                object
releases                object
contributors            object
issues_open             object
issues_closed           object
pull_requests_open      object
pull_requests_closed    object
lang_used               object
dtype: object

In [110]:
df.fillna(value=np.nan, inplace=True)

In [111]:
### Convert data types to numeric
numeric_columns = ['stars','forks','watchers', 'dependents', 'commits', 'branches'
                     , 'packages', 'releases', 'contributors', 'issues_open', 
                   'issues_closed', 'pull_requests_open', 'pull_requests_closed']

for num_col in numeric_columns:
    df[num_col] = df[num_col].str.replace('.','')
    df[num_col] = df[num_col].str.replace('m','00000')
    df[num_col] = pd.to_numeric(df[num_col])

In [112]:
# Create Language df
df_lang = pd.DataFrame.from_records(df['lang_used'])

In [113]:
# Merge into one
df = pd.concat([df, df_lang], axis=1)

In [114]:
df = df.drop(['lang_used'], axis=1)

In [136]:
# Export
df_1.to_csv('github_data1.csv',index=False)

In [119]:
# Export
df_1 = pd.read_csv('github_data1.csv')
df_2 = pd.read_csv('github_data2.csv')
df_3 = pd.read_csv('github_data3.csv')

In [131]:
df_1.shape

(329, 88)

In [126]:
df_2.shape

(200, 85)

In [133]:
df_3.shape

(57, 56)

In [134]:
df = pd.concat([df_1, df_2, df_3],axis=0)
df.shape

(586, 113)

In [135]:
df.fillna(value=0, inplace=True)
df.tail()

,collection,author,repository,stars,forks,watchers,dependents,commits,branches,packages,...,YASnippet,Inno Setup,Reason,Common Lisp,PureBasic,QMake,Smalltalk,SWIG,AngelScript,Liquid
52,Static Site Generators,react-static,react-static,8715,734.0,129,2200.0,1541,7,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,Static Site Generators,docsify,docsifyjs,13807,3233.0,227,1100.0,1286,13,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,Static Site Generators,mkdocs,mkdocs,10156,1489.0,227,6600.0,1457,6,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,Static Site Generators,harp,sintaxi,4868,377.0,115,0.0,638,83,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56,Static Site Generators,netlify-cms,netlify,11841,1774.0,270,41200.0,2853,34,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
df.to_csv('github_data_full.csv',index=False)

## Vývojová část
Tato část je věnována vývoji skriptu. Procházení stráchkách, zkoumání výstupu a podobně.

In [22]:
## Login
driver.get('https://github.com/login?return_to=%2Fcollections')
login_input = driver.find_element_by_id("login_field")
login_input.send_keys('')

password_input = driver.find_element_by_id("password")
password_input.send_keys('')

submit_btn = driver.find_element_by_name("commit")
submit_btn.click()

In [18]:
driver.get('https://github.com/collections')

In [ ]:
collections = driver.find_elements_by_tag_name('article')

#for collection in collections:
#    collection_link = collection.find_element_by_tag_name('a')
#    print(collection_link.text)

In [ ]:
# Collection list
collection = driver.find_elements_by_tag_name('article')[1]

# Get title of collection
collection_link = collection.find_element_by_tag_name('a')
collection_title = collection_link.text

# Click through link
collection_link.click()

In [ ]:
if(driver.find_element_by_xpath("//button[contains(@data-disable-with,'Loading more')]")):
    driver.find_element_by_xpath("//button[contains(@data-disable-with,'Loading more')]").click()

### Repo list - start, forks

In [ ]:
# Repository list
repo_list_item = driver.find_elements_by_tag_name('article')[2]
repo_list_link = repo_list_item.find_element_by_tag_name('a')
repo_list_title = repo_list_link.text

# Get title of collection
num_stars = repo_list_item.find_elements_by_xpath("//*//div//child::a[contains(@data-ga-click,'location:collection')]")[0].text
num_forks = repo_list_item.find_elements_by_xpath("//div/a[contains(@data-ga-click,'location:collection')]")[1].text

for repo_a in repo_list_item.find_elements_by_tag_name('a'):
    if('stargazers' in repo_a.get_attribute('href').lower()):
        num_stars=repo_a.text

    if('members' in repo_a.get_attribute('href').lower()):
        num_forks=repo_a.text
                    
# Click through link
repo_list_link.click()
repo_list_title

In [ ]:
driver.execute_script("arguments[0].click();", repo_list_link)

In [ ]:
num_stars

### Repo Name

In [ ]:
repo_name = driver.find_element_by_xpath("//strong[contains(@itemprop,'name')]").text
repo_author = driver.find_element_by_xpath("//span[contains(@itemprop,'author')]").text

### Watchers

In [ ]:
page_head = driver.find_element_by_class_name('pagehead-actions')
watch = page_head.find_elements_by_tag_name('li')[0]
num_watchers_dirty = watch.find_elements_by_tag_name('a')[1].text
num_watchers = num_watchers_dirty.replace('.','').replace('k','00')

### Lang usage

In [ ]:
lang_usage_dict = {}
#driver.find_element_by_class_name('details-reset').click()
lang_bar = driver.find_element_by_class_name('repository-lang-stats-graph')
lang_stats = lang_bar.find_elements_by_tag_name('span')

for lang_item in lang_stats:
    lang_info = lang_item.get_attribute('aria-label')
    lang_name = ["".join(x) for _, x in itertools.groupby(lang_info, key=str.isdigit)][0].strip()
    lang_usage = ["".join(x) for _, x in itertools.groupby(lang_info, key=str.isdigit)][1].replace('%','')
    lang_usage = float(lang_usage)
    lang_usage_dict[lang_name] = lang_usage
    
lang_usage_dict

### Summary bar

In [ ]:
# Summary Bar
summary_bar = driver.find_element_by_class_name('numbers-summary')
commmit = summary_bar.find_elements_by_tag_name('li')[0]
branches = summary_bar.find_elements_by_tag_name('li')[1]
packages = summary_bar.find_elements_by_tag_name('li')[2]
releases = summary_bar.find_elements_by_tag_name('li')[3]
contributors = summary_bar.find_elements_by_tag_name('li')[4]

# Commits
num_commits = commmit.find_element_by_class_name('num')
# Branches
num_branches = branches.find_element_by_class_name('num')
# Packages
num_packages = packages.find_element_by_class_name('num')
# Releases
num_releases = releases.find_element_by_class_name('num')
# Contributors
num_contributors = contributors.find_element_by_class_name('num')

### Issues

In [48]:
issues_link = driver.find_element_by_xpath("//a[contains(@data-selected-links,'repo_issues')]")
issues_link.click()

In [ ]:
## Issues
issues = driver.find_elements_by_css_selector(".table-list-header-toggle")[0]
issues_open = issues.find_elements_by_tag_name('a')[0].get_attribute('textContent')
issues_open_num = ''.join(re.findall(r'\d+', issues_open))

# Issues Closed
issues_closed = issues.find_elements_by_tag_name('a')[1].get_attribute('textContent')
issues_closed_num = ''.join(re.findall(r'\d+', issues_closed))

### Pull requests

In [ ]:
pull_request_link = driver.find_element_by_xpath("//a[contains(@data-selected-links,'repo_pulls')]")
pull_request_link.click()

In [ ]:
## Pull requests
pull_request = driver.find_elements_by_css_selector(".table-list-header-toggle")[0]
pull_request_open = pull_request.find_elements_by_tag_name('a')[0].get_attribute('textContent')
pull_request_open_num = ''.join(re.findall(r'\d+', pull_request_open))

# Pull requests Closed
pull_request_closed = pull_request.find_elements_by_tag_name('a')[1].get_attribute('textContent')
pull_request_closed_num = ''.join(re.findall(r'\d+', pull_request_closed))

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Firefox()
driver.get("http://somedomain/url_that_delays_loading")
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "myDynamicElement"))
    )
finally:
    driver.quit()